In [252]:
import numpy as np

In [319]:
Distance = [[0,1064,673,1401,277],
            [1064,0,958,1934,377],
            [673,957,0,1001,399],
            [1401,1934,1001,0,387],
            [277,377,399,387,0]]
W_ = {1:{1:'A',2:'E',3:'D',4:'C',5:'A'},
        2:{1:'B',2:'E',3:'A',4:'C',5:'D'},
        3:{1:'B',2:'A',3:'B',4:'C',5:'E'},
        4:{1:'D',2:'A',3:'D',4:'B',5:'D'},
        5:{1:'B',2:'E',3:'C',4:'B',5:'D'}}
Starting_Point = ['A','B','C','D','E']
fac_dict = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4}

MP2
===
You are the owner of a smart manufacturing company, with factories in five different cities: Atlanta, Boston, Chicago, Denver, and Edmonton. Each factory makes a different type of component: for convenience, let's call the components A, B, C, D, and E. The locations of your factories are a bit inconvenient to one another; the following table gives the distance (in miles) from one factory to another.

Distance | A | B | C | D | E
---------|---|---|---|---|---
A| 0 | 1064| 673 | 1401 | 277
B|1064| 0|	958|	1934|	337
C|673|958|0|	    1001|	    399
D|1401| 1934| 1001	|0|387
E|277|	337|	399|	387|	0
            
---
From those components, you manufacture five different types of widgets. Each widget is composed of five widgets of different types, as follows:

* Widget 1 = Components AEDCA
* Widget 2 = Components BEACD
* Widget 3 = Components BABCE
* Widget 4 = Components DADBD
* Widget 5 = Components BECBD
---
The goal of this assignment is to implement A* search that will find two different factory sequences:

* First, find the factory sequence with the smallest number of stops (the smallest number of factories visited).
* Second, find the factory sequence with the smallest number of miles traveled.

Solve this problem using A* search. Your report should specify your state representation, actions, and your heuristic. The heuristic need not be very smart, but it needs to be non-zero, and it needs to be admissible and consistent.

In [320]:
class node():
    def __init__(self, cur_factory,last_table, gscore=float('inf'),fscore=float('inf'), finished_factory=set(), parent=''):
        self.gscore = gscore
        self.last_table = last_table
        self.cur_factory = cur_factory
        self.parent = parent
        self.fscore = fscore
        self.finished_factory = finished_factory

In [313]:
def h_min_dis(node,factory):
    table = node.last_table.copy()
    for num, i in enumerate (table):
        if i[0] == 1:
            if W_[num+1][1] == factory: i[0] = i[0] - 1
            continue
        elif i[1] == 1:
            if W_[num+1][2] == factory: i[1] = i[1] - 1 
            continue
        elif i[2] == 1:
            if W_[num+1][3] == factory: i[2] = i[2] - 1 
            continue
        elif i[3] == 1:
            if W_[num+1][4] == factory: i[3] = i[3] - 1
            continue    
        elif i[4] == 1:
            if W_[num+1][5] == factory: i[4] = i[4] - 1 
            continue
    fscore = Distance[fac_dict[node.cur_factory]][fac_dict[factory]]
    return table,fscore
    
def h_min_step(node,factory):
    '''
    calculate the huristic of a certain step of node
    '''
    table = node.last_table.copy()
    for num, i in enumerate (table):
        if i[0] == 1:
            if W_[num+1][1] == factory: i[0] = i[0] - 1
            continue
        elif i[1] == 1:
            if W_[num+1][2] == factory: i[1] = i[1] - 1 
            continue
        elif i[2] == 1:
            if W_[num+1][3] == factory: i[2] = i[2] - 1 
            continue
        elif i[3] == 1:
            if W_[num+1][4] == factory: i[3] = i[3] - 1
            continue    
        elif i[4] == 1:
            if W_[num+1][5] == factory: i[4] = i[4] - 1 
            continue
    fscore = table.sum()
    return table,fscore
            
def goal_test(node):
    if node.fscore == 0: return True
    return False

```python
function A*(start, goal)
    // The set of nodes already evaluated
    closedSet := {}

    // The set of currently discovered nodes that are not evaluated yet.
    // Initially, only the start node is known.
    openSet := {start}

    // For each node, which node it can most efficiently be reached from.
    // If a node can be reached from many nodes, cameFrom will eventually contain the
    // most efficient previous step.
    cameFrom := an empty map

    // For each node, the cost of getting from the start node to that node.
    gScore := map with default value of Infinity

    // The cost of going from start to start is zero.
    gScore[start] := 0

    // For each node, the total cost of getting from the start node to the goal
    // by passing by that node. That value is partly known, partly heuristic.
    fScore := map with default value of Infinity

    // For the first node, that value is completely heuristic.
    fScore[start] := heuristic_cost_estimate(start, goal)

    while openSet is not empty
        current := the node in openSet having the lowest fScore[] value
        if current = goal
            return reconstruct_path(cameFrom, current)

        openSet.Remove(current)
        closedSet.Add(current)

        for each neighbor of current
            if neighbor in closedSet
                continue		// Ignore the neighbor which is already evaluated.

            if neighbor not in openSet	// Discover a new node
                openSet.Add(neighbor)
            
            // The distance from start to a neighbor
            //the "dist_between" function may vary as per the solution requirements.
            tentative_gScore := gScore[current] + dist_between(current, neighbor)
            if tentative_gScore >= gScore[neighbor]
                continue		// This is not a better path.

            // This path is the best until now. Record it!
            cameFrom[neighbor] := current
            gScore[neighbor] := tentative_gScore
            fScore[neighbor] := gScore[neighbor] + heuristic_cost_estimate(neighbor, goal) 

    return failure

function reconstruct_path(cameFrom, current)
    total_path := [current]
    while current in cameFrom.Keys:
        current := cameFrom[current]
        total_path.append(current)
    return total_path
```

In [322]:
def A_star(start,hu):
    hu_table = np.ones((5,5))
    initnode = node(start,hu_table,gscore=0,fscore=25.0)
    table,fscore = hu(initnode,start)
    start_node = node(start,table,gscore=0,fscore=fscore,parent=start)
    close_set = set()
    open_set = set()
    open_set.add(start_node)
    cities = ['A','B','C','D','E']
    while len(open_set)!=0:
        min_huris = float('inf')
        for nodes in open_set:
            if nodes.fscore<min_huris:
                temp = nodes
        cur_node = temp          
        if goal_test(cur_node): return cur_node.parent
        close_set.add(cur_node)
        open_set.remove(cur_node)
        for city in cities:
            if city == cur_node.cur_factory: continue
            table,fscore = hu(cur_node,city)
            if fscore == cur_node.fscore: continue
            new_node = node(city,table,gscore=cur_node.gscore+1,fscore=fscore,parent=cur_node.parent+city)
            open_set.add(new_node)
    return

In [323]:
for t in Starting_Point:
    lenn = 1000
    print('for starting =',t)
    for i in range(100):
        retv = A_star(t,h_min_dis)
        if (len(retv) < lenn):
            lenn = len(retv)
            temp = retv
    print('temp=',temp)
    print('len=',lenn)
    print()

for starting = A
temp= A
len= 1

for starting = B
temp= B
len= 1

for starting = C
temp= C
len= 1

for starting = D
temp= D
len= 1

for starting = E
temp= E
len= 1



In [318]:
Starting_Point

['A', 'B', 'D']